In [1]:
import numpy as numpy
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
train_data = pd.read_csv('/kaggle/input/commonlitreadabilityprize/train.csv')
test_data = pd.read_csv('/kaggle/input/commonlitreadabilityprize/test.csv')

In [ ]:
vectoriser = TfidfVectorizer()
x = vectoriser.fit_transform(train_data['excerpt'].values)
# x should be 2834 x 26833 i.e. 26833 words, too many to deal with

In [ ]:
print(vectoriser.get_feature_names()) 
# gets vector like on slide 46, removes punctuation but not other stuff like numbers 
# so use the below to limit to just words and only get top 2000 words
# vectoriser converts to lowercase by default

vectoriser = TfidfVectorizer(stop_words = 'english', max_features = 2000, lowercase = True)
x = vectoriser.fit_transform(train_data['excerpt'].values)

# BUT above is wrong, shouldn't be vectorising on whole training set, should have only fitted on X_train and at the end, train on entire dataset - leads to data leakage, validation data is in vectorising stage here

In [ ]:
y = train_data.loc[:,'target']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
print(train_data[excerpt]['0']) # shows first paragraph
data = vectoriser.transform(train_data[excerpt][['0']]).toarray() # vectoriser expects a list
pd.DataFrame(data).T.plot()

## Create ML model

In [ ]:
from sklearn.neural_network import MLPRegressor

In [ ]:
regr = MLPRegressor(hidden_layer_size=(512,1024,512))
regr.fit(X_train, y_train) # might need to do X_train.toarray()
regr.predict(X_test)

In [ ]:
# check performance
regr.score(X_train, y_train)
regr.score(X_test, y_test) # likely lower i.e. overfitting

In [ ]:
import matplotlib.pyplot as plt

training_preds = regr.predict(X_train)
plt.plot(y_train, training_preds, 'bo') # real vs predicted label

test_preds = regr.predict(X_test)
plt.plot(y_test, test_preds, 'bo') # will be worse than training (more scattered)

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_train, training_preds)
mean_squared_error(y_test, test_preds)

In [ ]:
# train one last time with entire dataset
regr.fit(X, y)
# train with actual test set
X_test = vectoriser.transorm(test_data['excerpt'])
test_preds = regr.predict(X_test)

In [ ]:
# for kaggle submission
predictions = pd.DataFrame()
predictions['id'] = test_df['id']
predictions['target'] = test_preds
predictions.to_csv('submission.csv', index=False)